# About Slurm
Slurm Workload Manager is an open source job scheduler, formerly known as the Simple Linux Utility for Resource Management. When you submit a job to the talapas cluster, Slurm is the software that decides where and when to send it and keeps track of whether it completes successfully or not. 

In [1]:
!sacct

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11617267     sys/dashb+      short       lcni          1    RUNNING      0:0 
11617267.ba+      batch                  lcni          1    RUNNING      0:0 
11617267.ex+     extern                  lcni          1    RUNNING      0:0 


'sacct' shows job information for the current user. Each job has a JobID and a name. Jobs may have separate steps (indicated by jobId.something), with their own names. The '+' signs mean the names are too long for the default column widths. We can use the format command to change that and specify what columns to include.

In [15]:
!sacct --format="JobID%15,JobName%30, Partition,Account,State"

          JobID                        JobName  Partition    Account      State 
--------------- ------------------------------ ---------- ---------- ---------- 
       11617267      sys/dashboard/sys/jupyter      short       lcni    RUNNING 
 11617267.batch                          batch                  lcni    RUNNING 
11617267.extern                         extern                  lcni    RUNNING 


We see I have one job: the OnDemand jupyterlab instance I'm using to create this very notebook. You can use this command to look for specific jobs by number, or jobs by other users.

In [10]:
!sacct -u dturnbul

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
9845296_294  utgovl_WW+       gc3f       gc3f          4    RUNNING      0:0 
9845296_294+     extern                  gc3f          4    RUNNING      0:0 
9845296_295  utgovl_WW+       gc3f       gc3f          4    RUNNING      0:0 
9845296_295+     extern                  gc3f          4    RUNNING      0:0 
9845296_298  utgovl_WW+       gc3f       gc3f          4    RUNNING      0:0 
9845296_298+     extern                  gc3f          4    RUNNING      0:0 


In [12]:
!sacct -j 11617266

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11617266     snakejob.+      short    kernlab          1  COMPLETED      0:0 
11617266.ba+      batch               kernlab          1  COMPLETED      0:0 
11617266.ex+     extern               kernlab          1  COMPLETED      0:0 


Back to our format command. There are a ton of columns you can include, and there's a special command to list them all.

In [14]:
!sacct --helpformat

Account             AdminComment        AllocCPUS           AllocGRES          
AllocNodes          AllocTRES           AssocID             AveCPU             
AveCPUFreq          AveDiskRead         AveDiskWrite        AvePages           
AveRSS              AveVMSize           BlockID             Cluster            
Comment             Constraints         ConsumedEnergy      ConsumedEnergyRaw  
CPUTime             CPUTimeRAW          DerivedExitCode     Elapsed            
ElapsedRaw          Eligible            End                 ExitCode           
Flags               GID                 Group               JobID              
JobIDRaw            JobName             Layout              MaxDiskRead        
MaxDiskReadNode     MaxDiskReadTask     MaxDiskWrite        MaxDiskWriteNode   
MaxDiskWriteTask    MaxPages            MaxPagesNode        MaxPagesTask       
MaxRSS              MaxRSSNode          MaxRSSTask          MaxVMSize          
MaxVMSizeNode       MaxVMSizeTask       

For completed jobs, "Elapsed"and "MaxRSS" are particularly helpful. They tell us how long the job ran, and how much memory it used. This can be used for planning how much time & memory to request in the future.

In [17]:
!sacct -j 11617266 --format="Elapsed, MaxRSS, ReqMem"

   Elapsed     MaxRSS     ReqMem 
---------- ---------- ---------- 
  00:11:44                 220Gc 
  00:11:44 107742656K      220Gc 
  00:11:44          0      220Gc 


# PIRGS again
All jobs run on talapas must include information about which account (pirg) should be billed for the time. If you have set the environment variables SLURM_ACCOUNT and SBATCH_ACCOUNT, it will use one of those. Otherwise you will need to specify it, so let's set that variable now (please use your own pirg, not LCNI!)

In [1]:
# what's my pirg again?
!groups

talapas lcni


In [1]:
mypirg = 'lcni' # USE YOUR OWN PLEASE

# Example data  
Let's take a look at the example data we will use from here on out. This is from openneuro: https://openneuro.org/datasets/ds000114/versions/1.0.1

In [1]:
bidsdir = '/projects/lcni/jolinda/shared/TalapasClass/ds000114/'

In [11]:
!tree {bidsdir} -L 1

/projects/lcni/jolinda/shared/TalapasClass/ds000114/
|-- CHANGES
|-- dataset_description.json
|-- dwi.bval
|-- dwi.bvec
|-- participants.tsv
|-- sub-01
|-- sub-02
|-- sub-03
|-- sub-04
|-- sub-05
|-- sub-06
|-- sub-07
|-- sub-08
|-- sub-09
|-- sub-10
|-- task-covertverbgeneration_bold.json
|-- task-covertverbgeneration_events.tsv
|-- task-fingerfootlips_bold.json
|-- task-fingerfootlips_events.tsv
|-- task-linebisection_bold.json
|-- task-overtverbgeneration_bold.json
|-- task-overtverbgeneration_events.tsv
|-- task-overtwordrepetition_bold.json
`-- task-overtwordrepetition_events.tsv

10 directories, 14 files


In [15]:
def pretty_print(jsonfile):
    import json
    with open(jsonfile) as f:
        j = json.load(f)
    print(json.dumps(j, indent = 4))

In [16]:
import os
pretty_print(os.path.join(bidsdir, 'dataset_description.json'))

{
    "Name": "A test-retest fMRI dataset for motor, language and spatial attention functions. ",
    "License": "",
    "Authors": [
        "Gorgolewski KJ",
        "Storkey A",
        "Bastin ME",
        "Whittle IR",
        "Wardlaw JM",
        "Pernet CR"
    ],
    "Acknowledgements": "",
    "HowToAcknowledge": "Please cite http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3641991/",
    "Funding": "",
    "ReferencesAndLinks": [
        "A test-retest fMRI dataset for motor, language and spatial attention functions, http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3641991/"
    ],
    "DatasetDOI": "10.18112/openneuro.ds000114.v1.0.1",
    "BIDSVersion": "1.0.0rc2"
}


In [12]:
!tree {bidsdir}/sub-01

/projects/lcni/jolinda/shared/TalapasClass/ds000114//sub-01
|-- ses-retest
|   |-- anat
|   |   `-- sub-01_ses-retest_T1w.nii.gz
|   |-- dwi
|   |   `-- sub-01_ses-retest_dwi.nii.gz
|   `-- func
|       |-- sub-01_ses-retest_task-covertverbgeneration_bold.nii.gz
|       |-- sub-01_ses-retest_task-fingerfootlips_bold.nii.gz
|       |-- sub-01_ses-retest_task-linebisection_bold.nii.gz
|       |-- sub-01_ses-retest_task-linebisection_events.tsv
|       |-- sub-01_ses-retest_task-overtverbgeneration_bold.nii.gz
|       `-- sub-01_ses-retest_task-overtwordrepetition_bold.nii.gz
`-- ses-test
    |-- anat
    |   `-- sub-01_ses-test_T1w.nii.gz
    |-- dwi
    |   `-- sub-01_ses-test_dwi.nii.gz
    `-- func
        |-- sub-01_ses-test_task-covertverbgeneration_bold.nii.gz
        |-- sub-01_ses-test_task-fingerfootlips_bold.nii.gz
        |-- sub-01_ses-test_task-linebisection_bold.nii.gz
        |-- sub-01_ses-test_task-linebisection_events.tsv
        |-- sub-01_ses-test_task-overtverbgenera

# Running a job: sbatch
You launch jobs from the command line with the sbatch command. Very simple jobs can be launched using the --wrap keyword. More complex jobs can be launched from a saved file.

In [5]:
!sbatch --account={mypirg} --wrap "echo hello"

Submitted batch job 11630227


In [6]:
cat slurm-11630227.out

hello


In [7]:
!sacct -j 11630227

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11630227           wrap      short       lcni          1  COMPLETED      0:0 
11630227.ba+      batch                  lcni          1  COMPLETED      0:0 
11630227.ex+     extern                  lcni          1  COMPLETED      0:0 


If we want a more descriptive name than "wrap", we can set that. There are lots of options you can set, here's a convenient list of them. https://slurm.schedmd.com/pdfs/summary.pdf

In [10]:
!sbatch --account={mypirg} --job-name='echo' --wrap "echo hello" 

Submitted batch job 11630274


In [11]:
!sacct -j 11630274

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11630274           echo      short       lcni          1  COMPLETED      0:0 
11630274.ba+      batch                  lcni          1  COMPLETED      0:0 
11630274.ex+     extern                  lcni          1  COMPLETED      0:0 


In many cases it's easier to write a script file and submit that. Here's a simple one.

In [17]:
cat fslinfo.srun

#!/bin/bash
#SBATCH --job-name=fslinfo
#SBATCH --account=lcni

module load fsl/6.0.1
fslinfo /home/jolinda/lcni/shared/TalapasClass/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz


# Parts of a slurm script  
  
First we define the interpreter. You'll probably usually want bash, but you can use other scripting languages such as python. Just make sure you have the right path  
`#!/bin/bash`   
  
Next up are our SLURM options, one per line. Account is the only required one (and that's only if you don't have environment variables set)  
`#SBATCH --job-name=fslinfo`   
`#SBATCH --account=lcni` 
   
Finally, we have our bash commands:  
`module load fsl/6.0.1`  
`fslinfo /home/jolinda/lcni/shared/TalapasClass/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz`

In [18]:
# submit it with sbatch
!sbatch fslinfo.srun

Submitted batch job 11638080


In [19]:
!sacct -j 11638080

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11638080        fslinfo      short       lcni          1  COMPLETED      0:0 
11638080.ba+      batch                  lcni          1  COMPLETED      0:0 
11638080.ex+     extern                  lcni          1  COMPLETED      0:0 


In [20]:
cat slurm-11638080.out

data_type	INT16
dim1		64
dim2		64
dim3		30
dim4		184
datatype	4
pixdim1		4.000000
pixdim2		4.000000
pixdim3		3.999975
pixdim4		2.500000
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+


# Slurm parameters to know (with examples
`--partition=long`  
Partition you are submitting to. The default is short.  
`--output=slurmjobname.out`  
`--error=slurmjobname-%j.err`  
Alternative to the slurm-{jobnumber}.out filename for standard out and standard error. %j will be replaced with the jobnumber  
`--time=5`  
`--time=5:30:0`  
`--time=2-0`  
Time limit for your job. First example is 5 minutes, second is 5 hours 30 minutes (h:m:s), third is 2 days (d-h). If unspecified it will be the limit for the partition (24 hours for short, 14 days for long, etc). Asking for less time means your job should launch more quickly. You'll want to set this if you are getting close to any planned talapas downtime, otherwise your job won't launch until after the cluster is back up.  
`--mem=16G`  
Memory requested for your job. The default amount varies, but it's about 4 GB for a standard node. This will not be enough for some jobs. You'll know you need more memory because your job will fail. Don't just ask for all the memory on the node (128 GB). It will take longer to launch your job and you will be taking resources away from other users. If you are doing something that requires more then 128 GB, use fat or longfat.  
`--cpus-per-task=1`  
Number of threads per task. If you are running something that can take advantage of multithreading, increase this number.  
`--dependency=afterok:123456`  
Run this job after job 123456 finishes with no errors  
`--mail-user=email_address -mail-type=END `  
Send an email when the job ends. I like to combine this with email-to-text from my carrier to get a text messsage notification.  
`--comment=idx:index`  
Write a comment. Use idx:index to indicate which project should be billed for your time.

# Array jobs  
We often need to run the same command on a long list of subjects. This is when array jobs come in handy. Instead of writing 10 different scripts with different subject names, we write one script with an array of names. Just separate out the part of the command that changes and replace it with ${x}, set a bash array variable named x, and include the array parameter in your file.

In [36]:
cat fslinfo_array.srun

#!/bin/bash
#SBATCH --job-name=fslinfo_array
#SBATCH --account=lcni
#SBATCH --array=0-9

data=(01 02 03 04 05 06 07 08 09 10)

x=${data[$SLURM_ARRAY_TASK_ID]}


module load fsl/6.0.1
echo ${x}
fslinfo /home/jolinda/lcni/shared/TalapasClass/ds000114/sub-${x}/ses-test/anat/sub-${x}_ses-test_T1w.nii.gz


In [37]:
!sbatch fslinfo_array.srun

Submitted batch job 11638382


In [38]:
!sacct -j 11638382

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
11638382_0   fslinfo_a+      short       lcni          1  COMPLETED      0:0 
11638382_0.+      batch                  lcni          1  COMPLETED      0:0 
11638382_0.+     extern                  lcni          1  COMPLETED      0:0 
11638382_1   fslinfo_a+      short       lcni          1  COMPLETED      0:0 
11638382_1.+      batch                  lcni          1  COMPLETED      0:0 
11638382_1.+     extern                  lcni          1  COMPLETED      0:0 
11638382_2   fslinfo_a+      short       lcni          1  COMPLETED      0:0 
11638382_2.+      batch                  lcni          1  COMPLETED      0:0 
11638382_2.+     extern                  lcni          1  COMPLETED      0:0 
11638382_3   fslinfo_a+      short       lcni          1  COMPLETED      0:0 
11638382_3.+      batch                  lcni          1  COMPLE

There's a slurm output file for each value of x.

In [40]:
ls slurm-11638382*

slurm-11638382_0.out  slurm-11638382_4.out  slurm-11638382_8.out
slurm-11638382_1.out  slurm-11638382_5.out  slurm-11638382_9.out
slurm-11638382_2.out  slurm-11638382_6.out
slurm-11638382_3.out  slurm-11638382_7.out


In [39]:
cat slurm-11638382_0.out

01
data_type	FLOAT32
dim1		256
dim2		156
dim3		256
dim4		1
datatype	16
pixdim1		1.000000
pixdim2		1.299376
pixdim3		1.000000
pixdim4		0.009668
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+


# Slurmpy
Writing similar scripts over and over got pretty old, so I wrote a slurm-script generator module named slurmpy. Using slurmpy you can create, submit, and see the results of slurm jobs. Here's how I could have generated today's examples.

In [2]:
import slurmpy

In [3]:
slurmpy.WrapSlurmCommand('echo hello', account = 'lcni')

sbatch --account=lcni --wrap "echo hello"
Submitted batch job 11649111



'11649111'

In [120]:
job = slurmpy.slurmjob(account = 'lcni', jobname = 'fslinfo')

In [121]:
import os
example_file = os.path.join(bidsdir, 'sub-01', 'ses-test', 'func', 
                            'sub-01_ses-test_task-fingerfootlips_bold.nii.gz')

In [122]:
job.AppendCommand('module load fsl/6.0.1')
job.AppendCommand('fslinfo ' + example_file)

In [123]:
job.WriteSlurmFile()

'fslinfo.srun'

In [124]:
job.PrintSlurmFile()

#!/bin/bash
#SBATCH --job-name=fslinfo
#SBATCH --account=lcni

module load fsl/6.0.1
fslinfo /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz


In [125]:
job.SubmitSlurmFile()

Submitted batch job 11649759



'11649759'

In [126]:
job.ShowStatus()

COMPLETED 1


In [127]:
job.ShowOutput()

slurm-11649759.out
data_type	INT16
dim1		64
dim2		64
dim3		30
dim4		184
datatype	4
pixdim1		4.000000
pixdim2		4.000000
pixdim3		3.999975
pixdim4		2.500000
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+



In [128]:
job.NewCommand()

In [129]:
subjects = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

In [130]:
job.ArrayJob(subjects)

In [131]:
job.AppendCommand('module load fsl/6.0.1')
job.AppendCommand('echo ${x}')
job.AppendCommand('fslinfo ' + bidsdir + '/sub-${x}/ses-test/anat/sub-${x}_ses-test_T1w.nii.gz')

In [136]:
job.AddSlurmParameters(time = 5)

In [137]:
job.WriteSlurmFile('fslinfo_array')

'fslinfo_array.srun'

In [138]:
job.PrintSlurmFile()

#!/bin/bash
#SBATCH --job-name=fslinfo_array
#SBATCH --account=lcni
#SBATCH --time=5
#SBATCH --array=0-9

data=(01 02 03 04 05 06 07 08 09 10)

x=${data[$SLURM_ARRAY_TASK_ID]}


module load fsl/6.0.1
echo ${x}
fslinfo /projects/lcni/jolinda/shared/TalapasClass/ds000114//sub-${x}/ses-test/anat/sub-${x}_ses-test_T1w.nii.gz


In [139]:
fslinfo_job.SubmitSlurmFile()

Submitted batch job 11649765



'11649765'

In [140]:
fslinfo_job.ShowStatus()

COMPLETED 10


In [142]:
fslinfo_job.ShowOutput(5)

slurm-11649765_5.out
06
data_type	FLOAT32
dim1		256
dim2		156
dim3		256
dim4		1
datatype	16
pixdim1		1.000000
pixdim2		1.300038
pixdim3		1.000000
pixdim4		0.009676
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+



# Job arrays in real life 
Today I typed out ['01', '02'] etc just to keep things straightforward, but of course we aren't really going to do all that work in real life, are we? Here are some alternatives.

In [149]:
# method one: generate a list of strings, zero padded, from 01 to 10:
['{:02d}'.format(x) for x in range(1,11)]

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

In [150]:
# method two: get all my subject names from the directory
import os
[x for x in os.listdir(bidsdir) if 'sub' in x]

['sub-03',
 'sub-08',
 'sub-02',
 'sub-07',
 'sub-01',
 'sub-06',
 'sub-05',
 'sub-04',
 'sub-09',
 'sub-10']

In [151]:
# if I want them in order
sorted([x for x in os.listdir(bidsdir) if 'sub' in x])

['sub-01',
 'sub-02',
 'sub-03',
 'sub-04',
 'sub-05',
 'sub-06',
 'sub-07',
 'sub-08',
 'sub-09',
 'sub-10']

In [152]:
# method 3: get all the filenames I want to run the command on, using glob
import glob
glob.glob(os.path.join(bidsdir, 'sub*', 'ses-test', 'anat', '*T1w.nii.gz'))

['/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-03/ses-test/anat/sub-03_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-08/ses-test/anat/sub-08_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-07/ses-test/anat/sub-07_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-06/ses-test/anat/sub-06_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-05/ses-test/anat/sub-05_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-04/ses-test/anat/sub-04_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-09/ses-test/anat/sub-09_ses-test_T1w.nii.gz',
 '/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-10/ses

In [153]:
# what our slurm file might look like with method 3
job.ArrayJob(glob.glob(os.path.join(bidsdir, 'sub*', 'ses-test', 'anat', '*T1w.nii.gz')))
job.NewCommand()
job.AppendCommand('module load fsl/6.0.1')
job.AppendCommand('echo ${x}')
job.AppendCommand('fslinfo ${x}')

In [154]:
job.WriteSlurmFile()

'fslinfo_array.srun'

In [155]:
job.PrintSlurmFile()

#!/bin/bash
#SBATCH --job-name=fslinfo_array
#SBATCH --account=lcni
#SBATCH --time=5
#SBATCH --array=0-9

data=(/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-03/ses-test/anat/sub-03_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-08/ses-test/anat/sub-08_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-07/ses-test/anat/sub-07_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-06/ses-test/anat/sub-06_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-05/ses-test/anat/sub-05_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-04/ses-test/anat/sub-04_ses-test_T1w.nii.gz /projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-09/ses-test/anat/sub-09_ses-test

In [156]:
job.SubmitSlurmFile()

Submitted batch job 11653464



'11653464'

In [160]:
job.ShowStatus()

COMPLETED 10


In [161]:
# can also see job info this way
print(job.JobInfo())

       JobID    JobName  Partition      State    Elapsed     MaxRSS 
------------ ---------- ---------- ---------- ---------- ---------- 
11653464_0   fslinfo_a+      short  COMPLETED   00:00:02            
11653464_0.+      batch             COMPLETED   00:00:02          0 
11653464_0.+     extern             COMPLETED   00:00:02          0 
11653464_1   fslinfo_a+      short  COMPLETED   00:00:02            
11653464_1.+      batch             COMPLETED   00:00:02          0 
11653464_1.+     extern             COMPLETED   00:00:03          0 
11653464_2   fslinfo_a+      short  COMPLETED   00:00:02            
11653464_2.+      batch             COMPLETED   00:00:02          0 
11653464_2.+     extern             COMPLETED   00:00:03          0 
11653464_3   fslinfo_a+      short  COMPLETED   00:00:02            
11653464_3.+      batch             COMPLETED   00:00:02          0 
11653464_3.+     extern             COMPLETED   00:00:03          0 
11653464_4   fslinfo_a+      short

In [162]:
job.ShowOutput()

slurm-11653464_0.out
/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-03/ses-test/anat/sub-03_ses-test_T1w.nii.gz
data_type	FLOAT32
dim1		256
dim2		156
dim3		256
dim4		1
datatype	16
pixdim1		1.000000
pixdim2		1.299594
pixdim3		1.000000
pixdim4		0.009672
cal_max		0.000000
cal_min		0.000000
file_type	NIFTI-1+

